In [1]:
import os
import pandas as pd
import numpy as np
import xml.dom.minidom as xmldom

# 本周主要工作——POI数据清理
### 使用osmpois.jar提取"university"和"college"

In [24]:
#筛选出全部POI
def extractPOI(PBF_PATH, EXTRACT_PATH):
    pbfdata_files = os.listdir(PBF_PATH)
    
    for pbf in pbfdata_files:
        if os.path.splitext(pbf)[1] == '.pbf':
            print(pbf)
            output_csv_fullname = os.path.join(EXTRACT_PATH ,pbf.replace('.osm.pbf','_all.csv'))
            TERMINAL_CODE1 = 'cd '+ PBF_PATH
            TERMINAL_CODE2 = 'java -jar osmpois.jar -of {} {}'.format(output_csv_fullname,pbf)
            os.system(TERMINAL_CODE1 + " && " + TERMINAL_CODE2)
            print(pbf + '--ALL POI EXTRACTED')

In [43]:
PBF_PATH = 'E:/AAA_Study/202003_GraduationProject/MapData/OSMData/sources'
EXTRACT_PATH = 'E:/AAA_Study/202003_GraduationProject/MapData/OSMData/POI_extract_all'

extractPOI(PBF_PATH,EXTRACT_PATH)

great_britain_latest.osm.pbf
great_britain_latest.osm.pbf--ALL POI EXTRACTED


In [48]:
#筛选出全部university和college的POI
def extractCampusPOI (INPUT_CSV_PATH, OUTPUT_CSV_PATH) :
    input_files = os.listdir(INPUT_CSV_PATH)
    for csv in input_files:
        input_csv_fullname = os.path.join(INPUT_CSV_PATH,csv)
        output_csv_fullname = os.path.join(OUTPUT_CSV_PATH,csv.replace('_all.csv','_campus.csv'))
        
        pois = pd.read_csv(input_csv_fullname,sep='|',header=None,low_memory=False).dropna()
        print(csv + '--ALL POI: '+ str(pois.shape))
        
        pois.index = range(pois.shape[0])
        pois.columns = ['type_code','OSM-ID','lat','lon','name']
        pois['type_code'] = pois['type_code'].values[:].astype(int)

        university_pois = pois[(pois['type_code'] == 22)]
        college_pois = pois[(pois['type_code'] == 19)]
        campus_pois = pd.concat([university_pois,college_pois])
        campus_pois.to_csv(output_csv_fullname,index=0,encoding='utf-8-sig')
        print(csv + '--CAMPUS POI EXTRACTED: '+ str(campus_pois.shape))

In [49]:
INPUT_CSV_PATH = 'E:/AAA_Study/202003_GraduationProject/MapData/OSMData/POI_extract_all'
OUTPUT_CSV_PATH = 'E:/AAA_Study/202003_GraduationProject/MapData/OSMData/POI_extract_campus'

extractCampusPOI(INPUT_CSV_PATH, OUTPUT_CSV_PATH)

china_latest_all.csv--ALL POI: (482022, 5)
china_latest_all.csv--CAMPUS POI EXTRACTED: (6316, 5)
great_britain_latest_all.csv--ALL POI: (900934, 5)
great_britain_latest_all.csv--CAMPUS POI EXTRACTED: (5660, 5)
switzerland-latest_all.csv--ALL POI: (147798, 5)
switzerland-latest_all.csv--CAMPUS POI EXTRACTED: (518, 5)
taiwan_latest_all.csv--ALL POI: (157279, 5)
taiwan_latest_all.csv--CAMPUS POI EXTRACTED: (1522, 5)


## 存在问题1——废数据：一些校门、教学楼、实验室等poi被一同提取
### 解决方法1：按照id访问原始osm文件，将tag不是university或college的数据剔除

##### 一个弊端：不得不将.pbf解压为.osm才能进行这种方法的筛选
##### China:580M->13G     Britain：1.1G->22G

In [54]:
#pbf转码为osm
def convert_pbf_osm (pbf_file) :
    base_path = 'E:/AAA_Study/202003_GraduationProject/'
    osmconvert_path = base_path + 'osmconvert'
    osm_file = pbf_file.replace('.osm.pbf','.osm')
    
    terminal_code = '{} {} > {}'.format(osmconvert_path,pbf_file,osm_file)
    os.system(terminal_code)
    print ('{} converted to {}'.format(pbf_file,osm_file))

In [171]:
OSMPBF_PATH = "E:/AAA_Study/202003_GraduationProject/MapData/OSMData/sources/great_britain_latest.osm.pbf"
convert_pbf_osm(OSMPBF_PATH)

E:/AAA_Study/202003_GraduationProject/MapData/OSMData/sources/great_britain_latest.osm.pbf converted to E:/AAA_Study/202003_GraduationProject/MapData/OSMData/sources/great_britain_latest.osm


In [3]:
#campus提取
def campus_extract(osmfilename):
    sel_osmkey = 'amenity'
    sel_osmvalue = ['college','university']
    
    osmfilter_path = 'E:/AAA_Study/202003_GraduationProject/' + 'osmfilter'
#     output_path = 'E:/AAA_Study/202003_GraduationProject/MapData/OSMData/education/'
    
    for value in sel_osmvalue:
        sel_element = sel_osmkey + "=" + value
        print (sel_element)
        output_sel = osmfilename.replace('.osm','_{}.osm'.format(value))
        print (output_sel)
        terminal_code = '{} {} --keep="{}" -o={}'.format(osmfilter_path,osmfilename,sel_element,output_sel)
        os.system(terminal_code)
        print ('{} extracted success!'.format(value)) 
    print("------education extracted success------")
   

In [4]:
OSM_PATH = "E:/AAA_Study/202003_GraduationProject/MapData/OSMData/sources/great_britain_latest.osm"
campus_extract(OSM_PATH)

amenity=college
E:/AAA_Study/202003_GraduationProject/MapData/OSMData/sources/great_britain_latest_college.osm
college extracted success!
amenity=university
E:/AAA_Study/202003_GraduationProject/MapData/OSMData/sources/great_britain_latest_university.osm
university extracted success!
------education extracted success------


In [3]:
#检查全部数据，筛除无用数据并获取bbox
def screenData (INPUT_CSV,INPUT_OSM,OUTPUT_CSV) :
    rawData = pd.read_csv(INPUT_CSV)
    print('raw university data:')
    print(rawData.shape)
    
    columns = rawData.columns.values #获取表头
    newDataStack = pd.DataFrame(columns=columns)
    bboxDataStack = []
    
    dom = xmldom.parse(INPUT_OSM)
    element = dom.documentElement
    sub_element_n = element.getElementsByTagName("node")
    sub_element_w = element.getElementsByTagName("way")
    
    #筛除无用数据：node数据和building……
    for ID in rawData['OSM-ID']:
        if 'W' in ID :
            real_id = ID.replace('W','')
            for i in range(len(sub_element_w)):
                if sub_element_w[i].getAttribute('id') == real_id:
                    way_tags = sub_element_w[i].getElementsByTagName("tag")
                    string = ''
                    for j in range(len(way_tags)):    
                        string = string + way_tags[j].getAttribute('k')+way_tags[j].getAttribute('v')
                    if 'university'or'college' in string and 'building' not in string:
                        line = rawData.loc[rawData['OSM-ID'] == ID].values
                        newDataStack = np.vstack((newDataStack,line))
                        
    #获取每个campus(way)的四边界经纬度……
    newData = pd.DataFrame(newDataStack,columns=columns)
    for newID in newData['OSM-ID']:
        way_id = newID[1:]
        temp_lat = []
        temp_lon = []
        for m in range(len(sub_element_w)):
            if sub_element_w[m].getAttribute('id') == way_id:
                way_nds = sub_element_w[m].getElementsByTagName("nd")
                for n in range(len(way_nds)):
                    node_id = way_nds[n].getAttribute('ref')
                    for p in range(len(sub_element_n)):
                        if sub_element_n[p].getAttribute('id') == node_id:
                            temp_lat.append(sub_element_n[p].getAttribute('lat'))
                            temp_lon.append(sub_element_n[p].getAttribute('lon'))
        bboxDataStack.append([max(temp_lat),min(temp_lat),max(temp_lon),min(temp_lon)])
                            
            
    bboxData = pd.DataFrame(bboxDataStack,columns=['north','south','east','west'])
    print('final university data:')
    print(newData.shape)
    newData.to_csv(OUTPUT_CSV,index=0,encoding='utf-8-sig')
    bboxData.to_csv(OUTPUT_CSV.replace('.csv','_bbox.csv'),index=0,encoding='utf-8-sig')
                        

In [4]:
INPUT_CSV = 'E:/AAA_Study/202003_GraduationProject/MapData/OSMData/POI_extract_campus/great_britain_latest_campus.csv'
INPUT_OSM = 'E:/AAA_Study/202003_GraduationProject/MapData/OSMData/education/great_britain_latest_university.osm'
OUTPUT_CSV = INPUT_CSV.replace('.csv','_new.csv')
screenData(INPUT_CSV,INPUT_OSM,OUTPUT_CSV)

raw university data:
(5660, 5)
final university data:
(891, 5)


### 根据整理过的校园POI保存建筑肌理数据（方法：bounding box）

In [2]:
import osmnx as ox
import networkx as nx

import csv
import time

In [ ]:
#仿照osmnx里的方法写一个footprints_from_bbox
def footprints_from_bbox(north,south,east,west, footprint_type='building', retain_invalid=False,
                          custom_settings=None):

    return ox.create_footprints_gdf(north=north, south=south, east=east, west=west,
                                 footprint_type=footprint_type, retain_invalid=retain_invalid,
                                 custom_settings=custom_settings)

In [8]:
def saveCampusBuildingFootprints(ID,lat,lon):
    name = ID[1:]
    coord = (float(lat),float(lon))
    gdf = ox.footprints.footprints_from_point(point=coord, distance=1000)
    if len(gdf) > 0:
        with open('E:/AAA_Study/202003_GraduationProject/zbzCode/json/'+name+'.json', 'w',encoding="utf-8") as f:
            f.write(gdf.to_json())
        gdf_proj = ox.project_gdf(gdf)
        bbox = ox.bbox_from_point(point=coord, distance=1000, project_utm=True)
        fig, ax = ox.footprints.plot_footprints(gdf_proj, bgcolor='#333333', color='w', figsize=(4,4), bbox=bbox,save=True, show=False, close=True, filename=name, dpi=100)
        print (name + ' fp:' + str(len(gdf)) + ' saved')
    else:
        print("gdf = 0 , save failed")

#     areas = gdf_proj.area




In [10]:
def test():
    starttime = time.time()
    CSV_FILE = 'E:/AAA_Study/202003_GraduationProject/MapData/OSMData/POI_extract_campus/great_britain_latest_campus_new1.csv'
    campus = []
    reader = csv.reader(open(CSV_FILE,encoding="utf-8"))
    for a, b, c, d, e in reader:
        colines = [b, c, d]  
        campus.append(colines)

    for i in range(len(campus)-1):
        saveCampusBuildingFootprints(campus[i+1][0],campus[i+1][1],campus[i+1][2])
    endtime = time.time()
    print('---------------------------')
    print('total time:', round(endtime - starttime, 2),'secs')

test()

145078273 fp:2159 saved
145602280 fp:2582 saved
146622787 fp:150 saved
146809173 fp:1984 saved
147020687 fp:1244 saved
147020691 fp:1227 saved
147148828 fp:353 saved
147488000 fp:3363 saved
148215569 fp:2334 saved
148617073 fp:1222 saved
149010150 fp:1184 saved
150498962 fp:523 saved
150513183 fp:994 saved
151180816 fp:2913 saved
151180818 fp:2509 saved
151180819 fp:2651 saved
151522438 fp:1488 saved
151609771 fp:1438 saved
151616519 fp:1904 saved
151911582 fp:2879 saved
151942966 fp:518 saved
152049535 fp:721 saved
152821326 fp:628 saved
153485385 fp:4015 saved
153585119 fp:1730 saved
153587836 fp:1635 saved
155024749 fp:136 saved
155223748 fp:1156 saved
155223867 fp:990 saved
156887742 fp:4141 saved
157556166 fp:2449 saved
157588898 fp:2415 saved
157592039 fp:1966 saved
158792688 fp:772 saved
159233691 fp:4513 saved
160470158 fp:86 saved
161195701 fp:5059 saved
161524919 fp:1660 saved
162207165 fp:3027 saved
162611061 fp:114 saved
162786368 fp:2302 saved
162847091 fp:2291 saved
16364

519177772 fp:1882 saved
528728566 fp:1094 saved
531307315 fp:1136 saved
537365487 fp:1373 saved
546360218 fp:2901 saved
554313566 fp:2127 saved
554315833 fp:1853 saved
555125116 fp:371 saved
555125118 fp:428 saved
556671020 fp:1805 saved
557609530 fp:1233 saved
557814832 fp:4104 saved
562564630 fp:787 saved
562566684 fp:22 saved
562566909 fp:21 saved
564120080 fp:1678 saved
566208787 fp:2396 saved
567866034 fp:565 saved
582932844 fp:2101 saved
584754108 fp:570 saved
586101510 fp:1187 saved
586101513 fp:1188 saved
586101514 fp:1214 saved
587122303 fp:3103 saved
588208386 fp:7441 saved
588672466 fp:149 saved
588683372 fp:94 saved
600818704 fp:387 saved
603440983 fp:97 saved
607971776 fp:1750 saved
618492044 fp:1391 saved
618555473 fp:712 saved
625416371 fp:162 saved
626293207 fp:64 saved
628161500 fp:1384 saved
628161799 fp:511 saved
628162035 fp:120 saved
628162981 fp:413 saved
630677547 fp:842 saved
630683993 fp:302 saved
631955842 fp:536 saved
641429750 fp:3018 saved
642492594 fp:1112